# Install necessary libraries

In [ ]:
!pip install -U funasr modelscope

# Record audio from mic

In [53]:
import numpy as np
from funasr import AutoModel
import pyaudio
import wave

In [ ]:
def record_audio(filename="custom_audio.wav", duration=3, fs=44100):
    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Open a stream
    stream = audio.open(format=pyaudio.paInt16, channels=1,
                        rate=fs, input=True,
                        frames_per_buffer=1024)

    print("Recording...")
    frames = []

    for _ in range(int(fs / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    print("Recording finished")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the audio file
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wf.setframerate(fs)
        wf.writeframes(b''.join(frames))

# Record a 3-second audio sample
record_audio()


In [ ]:
model = AutoModel(model="iic/emotion2vec_base_finetuned") # Alternative: iic/emotion2vec_plus_seed, iic/emotion2vec_plus_base, iic/emotion2vec_plus_large and iic/emotion2vec_base_finetuned

# Make predictions on audio files

In [68]:
# ask for the path of the audio file
wav_file = input("Enter the path of the audio file: ")

In [ ]:
rec_result = model.generate(wav_file, output_dir="./outputs", granularity="utterance", extract_embedding=False)

# print all labels and scores
for label, score in zip(rec_result[0]["labels"], rec_result[0]["scores"]):
    if label == "<unk>":
        print(f"Unknown: {score}")
    else:
        print(f"{label.split('/')[1]}: {score}")

# print the predicted label - might be buggy sometimes
predicted_label = rec_result[0]["labels"][np.argmax(rec_result[0]["scores"])]
print(f"\nPredicted Label: {predicted_label.split('/')[1]}")